# 📊 Comprehensive Policy Benchmarking

This notebook performs **fair, unbiased comparisons** between all policies:

| Policy Type | Policies |
|-------------|----------|
| **RL Models** | Model 1 (Levels 1-3), Model 2 (Extreme) |
| **Classical** | TBS, BaseStock, DoNothing |
| **Advanced** | PIL, DIP, PEIP, VectorBS |

**Methodology:**
- All policies evaluated on the **exact same environment instances**
- Same random seeds for fair demand/spoilage realization
- Comprehensive metrics: cost, fill rate, **spoilage rate**, orders

---

## 1️⃣ Setup & Imports

In [ ]:
# Install dependencies (run once)
# !pip install stable-baselines3 gymnasium numpy pandas matplotlib seaborn scipy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
import json
import warnings
warnings.filterwarnings('ignore')

# RL imports
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from gymnasium.wrappers import TimeLimit

# Project imports
import sys
sys.path.insert(0, '..')

from colab_training.gym_env import PerishableInventoryGymWrapper, RewardConfig
from colab_training.environment_suite import (
    get_canonical_suite, build_environment_from_config
)
from colab_training.benchmark import (
    get_tbs_policy_for_env, get_basestock_policy_for_env,
    get_pil_policy_for_env, get_dip_policy_for_env,
    get_peip_policy_for_env, get_vector_bs_policy_for_env,
    get_all_policies_for_env, AVAILABLE_BASELINES
)
from perishable_inventory_mdp.policies import DoNothingPolicy

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11

# All policies we will benchmark
ALL_POLICIES = ['RL', 'TBS', 'BaseStock', 'DoNothing', 'PIL', 'DIP', 'PEIP', 'VectorBS']

print('✅ All imports successful!')
print(f'Available baselines: {AVAILABLE_BASELINES}')
print(f'Policies to benchmark: {ALL_POLICIES}')

## 2️⃣ Configuration

Configure your RL model paths and evaluation parameters.

In [ ]:
# ============================================================
# CONFIGURATION - EDIT THESE PATHS TO YOUR MODELS
# ============================================================

# RL Model paths
RL_MODEL_LEVELS_1_3 = 'logs/best_model/best_model.zip'  # For simple, moderate, complex
RL_MODEL_EXTREME = 'logs/extreme_model/best_model.zip'  # For extreme level

# Evaluation parameters
N_EPISODES = 10          # Episodes per environment (increase for more accuracy)
N_ENVS_PER_LEVEL = 10    # Environments per complexity level
MAX_STEPS = 500          # Max steps per episode
RANDOM_SEED = 42         # For reproducibility

# Output directory
OUTPUT_DIR = Path('benchmark_results')
OUTPUT_DIR.mkdir(exist_ok=True)

print(f'📁 Results will be saved to: {OUTPUT_DIR.absolute()}')

## 3️⃣ Load RL Models

In [ ]:
# Load RL models
print('📦 Loading RL Models...')

try:
    model_levels_1_3 = PPO.load(RL_MODEL_LEVELS_1_3)
    print(f'  ✅ Model (Levels 1-3): {RL_MODEL_LEVELS_1_3}')
except FileNotFoundError:
    model_levels_1_3 = None
    print(f'  ⚠️ Model not found: {RL_MODEL_LEVELS_1_3}')

try:
    model_extreme = PPO.load(RL_MODEL_EXTREME)
    print(f'  ✅ Model (Extreme): {RL_MODEL_EXTREME}')
except FileNotFoundError:
    model_extreme = None
    print(f'  ⚠️ Model not found: {RL_MODEL_EXTREME}')

if model_levels_1_3 is None and model_extreme is None:
    print('\n⚠️ WARNING: No RL models found! Will benchmark baselines only.')

## 4️⃣ Environment Setup

In [ ]:
# Load environment suite
suite = get_canonical_suite()
print(f'📊 Loaded {len(suite)} environments')

# Display complexity distribution
complexity_counts = {}
for level in ['simple', 'moderate', 'complex', 'extreme']:
    configs = suite.get_by_complexity(level)
    complexity_counts[level] = len(configs)
    print(f'  {level.upper()}: {len(configs)} environments')

# Reward configuration
reward_config = RewardConfig()

def create_env(config, seed=None):
    '''Create gym environment from config with optional seed.'''
    mdp = build_environment_from_config(config)
    env = PerishableInventoryGymWrapper(mdp=mdp, reward_config=reward_config)
    env = TimeLimit(env, max_episode_steps=MAX_STEPS)
    env = Monitor(env)
    if seed is not None:
        env.reset(seed=seed)
    return env

## 5️⃣ Evaluation Framework

Fair evaluation: all policies tested on **identical** environment instances.

In [ ]:
@dataclass
class EpisodeMetrics:
    '''Metrics from a single episode.'''
    total_cost: float
    holding_cost: float
    backorder_cost: float
    spoilage_cost: float
    ordering_cost: float
    total_demand: float
    total_sales: float
    total_spoilage: float
    total_orders: float
    total_arrivals: float  # NEW: for spoilage rate calculation
    fill_rate: float
    spoilage_rate: float    # NEW: spoilage rate metric
    avg_inventory: float

@dataclass
class PolicyResult:
    '''Aggregated results for a policy.'''
    policy_name: str
    env_id: str
    complexity: str
    episodes: List[EpisodeMetrics] = field(default_factory=list)
    
    @property
    def mean_cost(self):
        return np.mean([e.total_cost for e in self.episodes])
    
    @property
    def std_cost(self):
        return np.std([e.total_cost for e in self.episodes])
    
    @property
    def mean_fill_rate(self):
        return np.mean([e.fill_rate for e in self.episodes])
    
    @property
    def mean_spoilage(self):
        return np.mean([e.total_spoilage for e in self.episodes])
    
    @property
    def mean_spoilage_rate(self):  # NEW
        return np.mean([e.spoilage_rate for e in self.episodes])

print('✅ Data classes defined')

In [ ]:
def run_episode(policy, env, is_rl_model=False):
    '''Run single episode and collect metrics including spoilage rate.'''
    obs, info = env.reset()
    done = False
    truncated = False
    
    total_cost = 0
    holding_cost = 0
    backorder_cost = 0
    spoilage_cost = 0
    ordering_cost = 0
    total_demand = 0
    total_sales = 0
    total_spoilage = 0
    total_orders = 0
    total_arrivals = 0  # Track arrivals for spoilage rate
    inventory_sum = 0
    steps = 0
    
    while not done and not truncated:
        if is_rl_model:
            action, _ = policy.predict(obs, deterministic=True)
        else:
            # Baseline policy - get MDP state from wrapper
            wrapper = env
            while hasattr(wrapper, 'env'):
                if hasattr(wrapper, 'current_state'):
                    break
                wrapper = wrapper.env
            
            if hasattr(wrapper, 'current_state') and wrapper.current_state is not None:
                state = wrapper.current_state
                mdp = wrapper.mdp
                action_dict = policy.get_action(state, mdp)
                # Convert dict action to array
                action = np.zeros(env.action_space.shape[0])
                if hasattr(wrapper, 'supplier_order') and hasattr(wrapper, 'supplier_action_bins'):
                    for i, sid in enumerate(wrapper.supplier_order):
                        qty = action_dict.get(sid, 0)
                        bins = wrapper.supplier_action_bins[sid]
                        action[i] = min(range(len(bins)), key=lambda x: abs(bins[x] - qty))
            else:
                action = np.zeros(env.action_space.shape[0])
        
        obs, reward, done, truncated, info = env.step(action)
        
        # Accumulate metrics from info
        holding_cost += info.get('holding_cost', 0)
        backorder_cost += info.get('shortage_cost', 0)
        spoilage_cost += info.get('spoilage_cost', 0)
        ordering_cost += info.get('procurement_cost', 0)
        total_demand += info.get('demand', 0)
        total_sales += info.get('sales', 0)
        total_spoilage += info.get('spoilage', 0)
        
        # Track orders/arrivals for spoilage rate
        orders = info.get('orders', {})
        if isinstance(orders, dict):
            total_orders += sum(orders.values())
        else:
            total_orders += info.get('total_order', 0)
        
        # Arrivals approximation (orders placed lead_time steps ago)
        # Using orders as proxy since arrivals = orders after lead time
        total_arrivals += info.get('arrivals', info.get('total_order', sum(orders.values()) if isinstance(orders, dict) else 0))
        
        inventory_sum += info.get('inventory', 0)
        
        total_cost -= reward  # reward is negative cost
        steps += 1
    
    fill_rate = total_sales / max(total_demand, 1e-6)
    avg_inventory = inventory_sum / max(steps, 1)
    
    # Calculate spoilage rate: spoiled / (arrivals or orders or inventory throughput)
    # Best proxy: spoiled / total_orders if arrivals not tracked
    spoilage_rate = total_spoilage / max(total_orders, total_sales + total_spoilage, 1e-6)
    
    return EpisodeMetrics(
        total_cost=total_cost,
        holding_cost=holding_cost,
        backorder_cost=backorder_cost,
        spoilage_cost=spoilage_cost,
        ordering_cost=ordering_cost,
        total_demand=total_demand,
        total_sales=total_sales,
        total_spoilage=total_spoilage,
        total_orders=total_orders,
        total_arrivals=total_arrivals,
        fill_rate=fill_rate,
        spoilage_rate=spoilage_rate,
        avg_inventory=avg_inventory
    )

print('✅ Episode runner ready')

In [ ]:
def evaluate_all_policies_fair(
    config,
    rl_model,
    n_episodes: int = 10,
    seed: int = 42
) -> Dict[str, PolicyResult]:
    '''
    Evaluate ALL 7+ policies on the SAME environment with SAME seeds.
    
    Policies: RL, TBS, BaseStock, DoNothing, PIL, DIP, PEIP, VectorBS
    '''
    results = {}
    env_id = config.env_id
    complexity = config.complexity
    
    # Create base environment for policy creation
    base_env = create_env(config, seed=seed)
    
    # Build policy dictionary with ALL policies
    policies = {}
    
    # RL model (if available)
    if rl_model is not None:
        policies['RL'] = (rl_model, True)
    
    # TBS
    try:
        policies['TBS'] = (get_tbs_policy_for_env(base_env), False)
    except Exception as e:
        print(f'    ⚠️ TBS unavailable: {e}')
    
    # BaseStock
    try:
        policies['BaseStock'] = (get_basestock_policy_for_env(base_env), False)
    except Exception as e:
        print(f'    ⚠️ BaseStock unavailable: {e}')
    
    # DoNothing (always available)
    policies['DoNothing'] = (DoNothingPolicy(), False)
    
    # PIL
    try:
        policies['PIL'] = (get_pil_policy_for_env(base_env), False)
    except Exception as e:
        print(f'    ⚠️ PIL unavailable: {e}')
    
    # DIP
    try:
        policies['DIP'] = (get_dip_policy_for_env(base_env), False)
    except Exception as e:
        print(f'    ⚠️ DIP unavailable: {e}')
    
    # PEIP
    try:
        policies['PEIP'] = (get_peip_policy_for_env(base_env), False)
    except Exception as e:
        print(f'    ⚠️ PEIP unavailable: {e}')
    
    # VectorBS
    try:
        policies['VectorBS'] = (get_vector_bs_policy_for_env(base_env), False)
    except Exception as e:
        print(f'    ⚠️ VectorBS unavailable: {e}')
    
    base_env.close()
    
    print(f'    Evaluating {len(policies)} policies: {list(policies.keys())}')
    
    # Evaluate each policy with SAME seeds for each episode
    for policy_name, (policy, is_rl) in policies.items():
        result = PolicyResult(policy_name=policy_name, env_id=env_id, complexity=complexity)
        
        for ep in range(n_episodes):
            episode_seed = seed + ep
            env = create_env(config, seed=episode_seed)
            
            try:
                metrics = run_episode(policy, env, is_rl_model=is_rl)
                result.episodes.append(metrics)
            except Exception as e:
                print(f'    ⚠️ {policy_name} failed episode {ep}: {e}')
            finally:
                env.close()
        
        results[policy_name] = result
    
    return results

print('✅ Fair comparison function ready')

## 6️⃣ Run Comprehensive Benchmarks

In [ ]:
# Run comprehensive benchmarks
all_results = []

print('=' * 70)
print('📊 COMPREHENSIVE POLICY BENCHMARKING')
print('=' * 70)
print(f'Episodes per env: {N_EPISODES}')
print(f'Environments per level: {N_ENVS_PER_LEVEL}')
print(f'Policies: {ALL_POLICIES}')
print('=' * 70)

for complexity in ['simple', 'moderate', 'complex', 'extreme']:
    configs = suite.get_by_complexity(complexity)
    n_envs = min(N_ENVS_PER_LEVEL, len(configs))
    
    if n_envs == 0:
        continue
    
    # Select appropriate RL model
    if complexity == 'extreme':
        rl_model = model_extreme
        model_label = 'RL (Extreme)'
    else:
        rl_model = model_levels_1_3
        model_label = 'RL (L1-3)'
    
    print(f'\n🔍 {complexity.upper()} ({n_envs} environments)')
    print(f'   Using: {model_label}')
    print('-' * 50)
    
    for i, config in enumerate(configs[:n_envs]):
        print(f'  [{i+1}/{n_envs}] {config.env_id}...')
        
        try:
            results = evaluate_all_policies_fair(
                config=config,
                rl_model=rl_model,
                n_episodes=N_EPISODES,
                seed=RANDOM_SEED
            )
            
            for policy_name, result in results.items():
                all_results.append({
                    'policy': policy_name,
                    'env_id': config.env_id,
                    'complexity': complexity,
                    'mean_cost': result.mean_cost,
                    'std_cost': result.std_cost,
                    'fill_rate': result.mean_fill_rate,
                    'spoilage': result.mean_spoilage,
                    'spoilage_rate': result.mean_spoilage_rate,  # NEW
                    'n_episodes': len(result.episodes)
                })
            
        except Exception as e:
            print(f'    FAILED: {e}')

print('\n' + '=' * 70)
print('✅ BENCHMARKING COMPLETE')
print('=' * 70)

# Create DataFrame
df = pd.DataFrame(all_results)
print(f'\nTotal evaluations: {len(df)}')
print(f'Policies evaluated: {df["policy"].unique().tolist()}')
df.head(10)

In [ ]:
# Save results
results_path = OUTPUT_DIR / 'benchmark_results.csv'
df.to_csv(results_path, index=False)
print(f'💾 Results saved to: {results_path}')

# Save as JSON too
df.to_json(OUTPUT_DIR / 'benchmark_results.json', orient='records', indent=2)

## 7️⃣ Comprehensive Visualizations

All metrics: **Cost**, **Fill Rate**, **Spoilage Rate** across ALL policies.

In [ ]:
# 1. Mean Cost by Policy and Complexity
fig, ax = plt.subplots(figsize=(16, 8))

pivot = df.pivot_table(values='mean_cost', index='policy', columns='complexity', aggfunc='mean')
cols = ['simple', 'moderate', 'complex', 'extreme']
pivot = pivot[[c for c in cols if c in pivot.columns]]

pivot.plot(kind='bar', ax=ax, width=0.8, edgecolor='white', linewidth=1.5)
ax.set_xlabel('Policy', fontsize=12, fontweight='bold')
ax.set_ylabel('Mean Total Cost', fontsize=12, fontweight='bold')
ax.set_title('📊 Mean Cost by Policy and Complexity Level (All Policies)', fontsize=14, fontweight='bold')
ax.legend(title='Complexity', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', fontsize=8, padding=2)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'cost_by_complexity.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 2. Fill Rate by Policy and Complexity  
fig, ax = plt.subplots(figsize=(16, 8))

pivot_fr = df.pivot_table(values='fill_rate', index='policy', columns='complexity', aggfunc='mean')
cols = ['simple', 'moderate', 'complex', 'extreme']
pivot_fr = pivot_fr[[c for c in cols if c in pivot_fr.columns]]

pivot_fr.plot(kind='bar', ax=ax, width=0.8, edgecolor='white', linewidth=1.5)
ax.set_xlabel('Policy', fontsize=12, fontweight='bold')
ax.set_ylabel('Mean Fill Rate', fontsize=12, fontweight='bold')
ax.set_title('📦 Fill Rate by Policy and Complexity Level (All Policies)', fontsize=14, fontweight='bold')
ax.legend(title='Complexity', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_ylim(0, 1.1)
ax.axhline(y=0.95, color='red', linestyle='--', alpha=0.5, label='95% Target')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'fillrate_by_complexity.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 3. Spoilage Rate by Policy and Complexity (NEW)
fig, ax = plt.subplots(figsize=(16, 8))

pivot_sr = df.pivot_table(values='spoilage_rate', index='policy', columns='complexity', aggfunc='mean')
cols = ['simple', 'moderate', 'complex', 'extreme']
pivot_sr = pivot_sr[[c for c in cols if c in pivot_sr.columns]]

pivot_sr.plot(kind='bar', ax=ax, width=0.8, edgecolor='white', linewidth=1.5, colormap='Reds')
ax.set_xlabel('Policy', fontsize=12, fontweight='bold')
ax.set_ylabel('Mean Spoilage Rate', fontsize=12, fontweight='bold')
ax.set_title('🗑️ Spoilage Rate by Policy and Complexity Level (All Policies)', fontsize=14, fontweight='bold')
ax.legend(title='Complexity', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.axhline(y=0.05, color='green', linestyle='--', alpha=0.5, label='5% Target')

for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', fontsize=8, padding=2)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'spoilage_rate_by_complexity.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 4. Box Plot of Cost Distribution
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

for idx, complexity in enumerate(['simple', 'moderate', 'complex', 'extreme']):
    ax = axes[idx // 2, idx % 2]
    subset = df[df['complexity'] == complexity]
    
    if len(subset) > 0:
        subset.boxplot(column='mean_cost', by='policy', ax=ax)
        ax.set_title(f'{complexity.upper()}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Policy')
        ax.set_ylabel('Mean Cost')
        plt.sca(ax)
        plt.xticks(rotation=45, ha='right')

plt.suptitle('📈 Cost Distribution by Complexity Level (All Policies)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'cost_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 5. Heatmaps of All Metrics
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Cost heatmap
pivot_cost = df.pivot_table(values='mean_cost', index='policy', columns='complexity', aggfunc='mean')
sns.heatmap(pivot_cost, annot=True, fmt='.0f', cmap='RdYlGn_r', ax=axes[0])
axes[0].set_title('Mean Cost (lower is better)', fontweight='bold')

# Fill rate heatmap
pivot_fr = df.pivot_table(values='fill_rate', index='policy', columns='complexity', aggfunc='mean')
sns.heatmap(pivot_fr, annot=True, fmt='.2%', cmap='RdYlGn', ax=axes[1])
axes[1].set_title('Fill Rate (higher is better)', fontweight='bold')

# Spoilage rate heatmap (NEW)
pivot_sr = df.pivot_table(values='spoilage_rate', index='policy', columns='complexity', aggfunc='mean')
sns.heatmap(pivot_sr, annot=True, fmt='.2%', cmap='RdYlGn_r', ax=axes[2])
axes[2].set_title('Spoilage Rate (lower is better)', fontweight='bold')

plt.suptitle('🔥 Policy Performance Heatmaps (All Metrics)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'performance_heatmaps.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 6. RL vs Best Baseline Comparison
summary = df.groupby(['complexity', 'policy'])['mean_cost'].mean().reset_index()

comparison_data = []
for complexity in ['simple', 'moderate', 'complex', 'extreme']:
    subset = summary[summary['complexity'] == complexity]
    if len(subset) == 0:
        continue
    
    rl_cost = subset[subset['policy'] == 'RL']['mean_cost'].values
    rl_cost = rl_cost[0] if len(rl_cost) > 0 else None
    
    baselines = subset[subset['policy'] != 'RL']
    if len(baselines) > 0:
        best_baseline = baselines.loc[baselines['mean_cost'].idxmin()]
        comparison_data.append({
            'complexity': complexity,
            'RL_cost': rl_cost,
            'best_baseline': best_baseline['policy'],
            'best_baseline_cost': best_baseline['mean_cost'],
            'improvement': (best_baseline['mean_cost'] - rl_cost) / best_baseline['mean_cost'] * 100 if rl_cost else None
        })

comparison_df = pd.DataFrame(comparison_data)
print('\n📊 RL vs Best Baseline Summary:')
print('=' * 70)
print(comparison_df.to_string(index=False))

if len(comparison_df) > 0 and comparison_df['RL_cost'].notna().any():
    fig, ax = plt.subplots(figsize=(14, 7))
    
    x = np.arange(len(comparison_df))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, comparison_df['RL_cost'].fillna(0), width, label='RL', color='#2ecc71')
    bars2 = ax.bar(x + width/2, comparison_df['best_baseline_cost'], width, label='Best Baseline', color='#3498db')
    
    ax.set_xlabel('Complexity Level', fontsize=12, fontweight='bold')
    ax.set_ylabel('Mean Cost', fontsize=12, fontweight='bold')
    ax.set_title('🏆 RL vs Best Baseline by Complexity', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(comparison_df['complexity'].str.upper())
    ax.legend()
    
    for i, (_, row) in enumerate(comparison_df.iterrows()):
        if row['improvement'] and pd.notna(row['improvement']):
            color = 'green' if row['improvement'] > 0 else 'red'
            ax.annotate(f"{row['improvement']:.1f}%", 
                       xy=(i, max(row['RL_cost'] or 0, row['best_baseline_cost']) + 50),
                       ha='center', fontsize=10, color=color, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'rl_vs_baseline.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# 7. Overall Policy Ranking with All Metrics
ranking = df.groupby('policy').agg({
    'mean_cost': ['mean', 'std'],
    'fill_rate': 'mean',
    'spoilage': 'mean',
    'spoilage_rate': 'mean'  # NEW
}).round(4)

ranking.columns = ['Mean Cost', 'Std Cost', 'Fill Rate', 'Spoilage (units)', 'Spoilage Rate']
ranking = ranking.sort_values('Mean Cost')

print('\n🏆 OVERALL POLICY RANKING (by Mean Cost)')
print('=' * 90)
print(ranking.to_string())

# Visualize ranking
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Cost ranking
colors = plt.cm.viridis(np.linspace(0, 0.8, len(ranking)))
axes[0].barh(ranking.index, ranking['Mean Cost'], color=colors, edgecolor='white', linewidth=1.5)
axes[0].set_xlabel('Mean Cost', fontsize=11, fontweight='bold')
axes[0].set_title('Cost (Lower = Better)', fontsize=12, fontweight='bold')

# Fill rate ranking
fr_sorted = ranking.sort_values('Fill Rate', ascending=True)
axes[1].barh(fr_sorted.index, fr_sorted['Fill Rate'], color=plt.cm.Greens(np.linspace(0.3, 0.9, len(ranking))), edgecolor='white')
axes[1].set_xlabel('Fill Rate', fontsize=11, fontweight='bold')
axes[1].set_title('Fill Rate (Higher = Better)', fontsize=12, fontweight='bold')
axes[1].axvline(x=0.95, color='red', linestyle='--', alpha=0.7)

# Spoilage rate ranking (NEW)
sr_sorted = ranking.sort_values('Spoilage Rate', ascending=False)
axes[2].barh(sr_sorted.index, sr_sorted['Spoilage Rate'], color=plt.cm.Reds(np.linspace(0.3, 0.9, len(ranking))), edgecolor='white')
axes[2].set_xlabel('Spoilage Rate', fontsize=11, fontweight='bold')
axes[2].set_title('Spoilage Rate (Lower = Better)', fontsize=12, fontweight='bold')
axes[2].axvline(x=0.05, color='green', linestyle='--', alpha=0.7)

plt.suptitle('🏆 Overall Policy Ranking - All Metrics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'policy_ranking_all_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 8. Radar Chart of Multi-Metric Performance
from math import pi

# Prepare data - include spoilage rate
metrics = ['fill_rate', 'mean_cost', 'spoilage_rate']
policy_means = df.groupby('policy')[metrics].mean()

# Normalize metrics (invert cost and spoilage_rate so higher is better)
normalized = policy_means.copy()
normalized['mean_cost'] = 1 - (normalized['mean_cost'] - normalized['mean_cost'].min()) / (normalized['mean_cost'].max() - normalized['mean_cost'].min() + 1e-6)
normalized['spoilage_rate'] = 1 - (normalized['spoilage_rate'] - normalized['spoilage_rate'].min()) / (normalized['spoilage_rate'].max() - normalized['spoilage_rate'].min() + 1e-6)

# Create radar chart
categories = ['Fill Rate', 'Cost Efficiency', 'Low Spoilage Rate']
N = len(categories)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

colors = plt.cm.tab10(np.linspace(0, 1, len(normalized)))

for idx, (policy, row) in enumerate(normalized.iterrows()):
    values = [row['fill_rate'], row['mean_cost'], row['spoilage_rate']]
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=policy, color=colors[idx])
    ax.fill(angles, values, alpha=0.1, color=colors[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_title('🎯 Multi-Metric Policy Comparison\n(Higher = Better)', fontsize=14, fontweight='bold', y=1.08)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'radar_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 8️⃣ Statistical Analysis

In [ ]:
# Statistical summary
from scipy import stats as scipy_stats

print('📊 STATISTICAL SUMMARY')
print('=' * 90)

# Summary statistics with all metrics
summary_stats = df.groupby('policy').agg({
    'mean_cost': ['count', 'mean', 'std', 'min', 'max'],
    'fill_rate': ['mean', 'std'],
    'spoilage_rate': ['mean', 'std']  # NEW
}).round(4)

print('\n📈 Summary Statistics (All Metrics):')
print(summary_stats.to_string())

# Pairwise comparisons with RL
print('\n\n📉 RL vs Baselines Statistical Tests:')
print('-' * 90)

rl_costs = df[df['policy'] == 'RL']['mean_cost'].values

if len(rl_costs) > 0:
    for policy in df['policy'].unique():
        if policy == 'RL':
            continue
        
        baseline_costs = df[df['policy'] == policy]['mean_cost'].values
        
        if len(baseline_costs) > 1 and len(rl_costs) > 1:
            min_len = min(len(rl_costs), len(baseline_costs))
            t_stat, p_value = scipy_stats.ttest_ind(rl_costs[:min_len], baseline_costs[:min_len])
            
            rl_mean = np.mean(rl_costs)
            baseline_mean = np.mean(baseline_costs)
            improvement = (baseline_mean - rl_mean) / baseline_mean * 100
            
            sig = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''
            
            print(f'{policy:12} | RL: {rl_mean:.1f} vs {baseline_mean:.1f} | '
                  f'Diff: {improvement:+.1f}% | p={p_value:.4f} {sig}')
else:
    print('No RL results available for comparison')

print('\nSignificance levels: * p<0.05, ** p<0.01, *** p<0.001')

## 9️⃣ Final Summary & Export

In [ ]:
# Final summary
print('\n' + '=' * 90)
print('📋 BENCHMARK SUMMARY REPORT')
print('=' * 90)

print(f'\n📊 Evaluation Statistics:')
print(f'   Total evaluations: {len(df)}')
print(f'   Policies tested: {df["policy"].nunique()} ({df["policy"].unique().tolist()})')
print(f'   Environments tested: {df["env_id"].nunique()}')
print(f'   Episodes per evaluation: {N_EPISODES}')

print(f'\n🏆 Policy Rankings:')
print('\n   BY COST (lower is better):')
rankings_cost = df.groupby('policy')['mean_cost'].mean().sort_values()
for i, (policy, cost) in enumerate(rankings_cost.items(), 1):
    print(f'   {i}. {policy}: {cost:.1f}')

print('\n   BY FILL RATE (higher is better):')
rankings_fr = df.groupby('policy')['fill_rate'].mean().sort_values(ascending=False)
for i, (policy, fr) in enumerate(rankings_fr.items(), 1):
    print(f'   {i}. {policy}: {fr:.2%}')

print('\n   BY SPOILAGE RATE (lower is better):')
rankings_sr = df.groupby('policy')['spoilage_rate'].mean().sort_values()
for i, (policy, sr) in enumerate(rankings_sr.items(), 1):
    print(f'   {i}. {policy}: {sr:.2%}')

print(f'\n📁 Output Files:')
for f in OUTPUT_DIR.glob('*'):
    print(f'   {f}')

print('\n' + '=' * 90)
print('✅ BENCHMARKING COMPLETE!')
print('=' * 90)